In [2]:
import os
import openai
import groq
import requests
import json
from dotenv import load_dotenv

In [3]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
personal_api_key = os.getenv("PERSONAL_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
# Initialize OpenAI client
client = openai.OpenAI(api_key=openai_api_key)

In [5]:
def encode_image_to_base64(image_path):
    import base64
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

#1 load the content of voice file to ai and translate to text and save to file
def transcribe_audio(audio_file_path: str):
    groq_client = groq.Groq(api_key=groq_api_key)
    response = groq_client.audio.transcriptions.create(
        file=(audio_file_path, open(audio_file_path, "rb")),
        language="pl",
        model="whisper-large-v3"
    )
    return response.text

def save_audio_link_to_file(link: str, out_path : str):
    audio_response = requests.get(link)
    with open(out_path, "wb") as file:
        file.write(audio_response.content)

In [58]:
if not os.path.exists('context_base.md'):
    context_base = "<FACTS>\n"
    context_base += "{\n"
    for file in os.listdir('../pliki_z_fabryki/facts'):
        with open(f'../pliki_z_fabryki/facts/{file}', 'r') as file:
            content = file.read()
            context_base += "{\n"
            context_base += file.name[3:] + " : "
            context_base += content
            context_base += "}\n"
    context_base += "}\n"        
    context_base += "</FACTS>\n"
    context_base += "<REPORTS>\n"
    context_base += "{\n"
    for file in os.listdir('../pliki_z_fabryki'):
        if file.endswith('.txt'):
            with open(f'../pliki_z_fabryki/{file}', 'r') as file:
                content = file.read()
            context_base += "{\n"
            context_base += file.name[3:] + " : "
            context_base += content
            context_base += "\n}\n"
    context_base += "}\n"
    context_base += "</REPORTS>\n"
    with open('context_base.md', 'w') as file:
        file.write(context_base)

In [67]:
with open('context.md', 'r') as file:
    context = file.read()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "system", "content": context},
              {"role": "user", "content": "Do każdego pliku raport wygeneruj słowa kluczowe w formie mianownika (czyli np. “sportowiec”, a nie “sportowcem”, “sportowców” itp."}]
)
answers = response.choices[0].message.content


In [68]:
print(answers)
answers_json = json.loads(answers.replace("```json\n", "").replace("```", "").replace("],","],\n"))
with open('answers.json', 'w', encoding='utf-8') as f:
    json.dump(answers_json, f, ensure_ascii=False, indent=4)

{
    "2024-11-12_report-00-sektor_C4.txt": "Aleksander, Ragowski, nauczyciel, jednostka, kontrola",
    "2024-11-12_report-01-sektor_A1.txt": "alarm, ruch, organiczny, zwierzyna, bezpieczny",
    "2024-11-12_report-02-sektor_A3.txt": "obszar, patrol, nocny, cichy, aktywność",
    "2024-11-12_report-03-sektor_A3.txt": "patrol, monitorowanie, czujniki, życie, organiczne, bez, rezultat",
    "2024-11-12_report-04-sektor_B2.txt": "patrol, zachodni, anomalia, norma, sektor, bezpieczny",
    "2024-11-12_report-05-sektor_C1.txt": "aktywnosć, organiczna, technologiczna, sensor, dźwiękowy, detektor, ruch",
    "2024-11-12_report-06-sektor_C2.txt": "sektor, północno-zachodni, spokojny, temperatura, ruch",
    "2024-11-12_report-07-sektor_C4.txt": "czujnik, dźwięk, ultradźwiękowy, sygnał, nadajnik, Barbara, Zawadzka, JavaScript, Python",
    "2024-11-12_report-08-sektor_A1.txt": "monitoring, obszar, patrolowy, brak, ruch, cisza",
    "2024-11-12_report-09-sektor_C2.txt": "patrol, peryferia, zach

In [69]:
answer_json = {
    "task": "dokumenty",
    "apikey": personal_api_key,
    "answer": answers_json
}

In [70]:
answer_url = "https://centrala.ag3nts.org/report"
answer_response = requests.post(answer_url, json=answer_json)
print(answer_response.request.body)  # Displays the request body

# Assuming `login_response` is the response object from the login request
print("Status Code:", answer_response.status_code)  # Displays the status code (e.g., 200)
print("Response Body:", answer_response.text.encode('utf-8'))       # Displays the response content as a string

b'{"task": "dokumenty", "apikey": "1400cbf0-b7dd-49ab-9342-6ad8fd26ba69", "answer": {"2024-11-12_report-00-sektor_C4.txt": "Aleksander, Ragowski, nauczyciel, jednostka, kontrola", "2024-11-12_report-01-sektor_A1.txt": "alarm, ruch, organiczny, zwierzyna, bezpieczny", "2024-11-12_report-02-sektor_A3.txt": "obszar, patrol, nocny, cichy, aktywno\\u015b\\u0107", "2024-11-12_report-03-sektor_A3.txt": "patrol, monitorowanie, czujniki, \\u017cycie, organiczne, bez, rezultat", "2024-11-12_report-04-sektor_B2.txt": "patrol, zachodni, anomalia, norma, sektor, bezpieczny", "2024-11-12_report-05-sektor_C1.txt": "aktywnos\\u0107, organiczna, technologiczna, sensor, d\\u017awi\\u0119kowy, detektor, ruch", "2024-11-12_report-06-sektor_C2.txt": "sektor, p\\u00f3\\u0142nocno-zachodni, spokojny, temperatura, ruch", "2024-11-12_report-07-sektor_C4.txt": "czujnik, d\\u017awi\\u0119k, ultrad\\u017awi\\u0119kowy, sygna\\u0142, nadajnik, Barbara, Zawadzka, JavaScript, Python", "2024-11-12_report-08-sektor_A1